In [1]:
!(pip install streamlit -q)

     |████████████████████████████████| 9.1 MB 27.6 MB/s 
     |████████████████████████████████| 111 kB 71.9 MB/s 
     |████████████████████████████████| 232 kB 65.4 MB/s 
     |████████████████████████████████| 181 kB 70.9 MB/s 
     |████████████████████████████████| 4.3 MB 50.6 MB/s 
     |████████████████████████████████| 78 kB 8.3 MB/s 
     |████████████████████████████████| 164 kB 70.1 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 133 kB 66.4 MB/s 
     |████████████████████████████████| 132 kB 72.1 MB/s 
     |████████████████████████████████| 793 kB 66.6 MB/s 
     |████████████████████████████████| 428 kB 71.4 MB/s 
     |████████████████████████████████| 381 kB 29.7 MB/s 
     |████████████████████████████████| 51 kB 6.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbclient 0.6.4 requ

In [2]:
!pip install yacs
!pip install googletrans==4.0.0-rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.4 MB/s 
     |████████████████████████████████| 1.3 MB 53.3 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 53 kB 2.5 MB/s 
     |████████████████████████████████| 65 kB 2.8 MB/s 
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17416 sha256=afdfe848f2125bc9c963381d6116bcf129f5a0fc6cafb6ead90c01d6ac1f7d5a
  Stored in directory: /root/.cache/pip/wheels/43/34/00/4fe71786ea6d12314b29037620c36d857e5d104ac2748bf82a
Successfully built googletrans


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%%writefile /content/exec.sh
#!/bin/sh
python /content/drive/MyDrive/SAMSLR/data-prepare/wholepose/demo.py
python /content/drive/MyDrive/SAMSLR/SL-GCN/data_gen/sign_gendata.py
python /content/drive/MyDrive/SAMSLR/SL-GCN/data_gen/gen_bone_data.py
python /content/drive/MyDrive/SAMSLR/SL-GCN/data_gen/gen_motion_data.py
python /content/drive/MyDrive/SAMSLR/SL-GCN/main.py --config /content/drive/MyDrive/SAMSLR/SL-GCN/config/sign/test/test_joint.yaml
python /content/drive/MyDrive/SAMSLR/SL-GCN/main.py --config /content/drive/MyDrive/SAMSLR/SL-GCN/config/sign/test/test_bone.yaml
python /content/drive/MyDrive/SAMSLR/SL-GCN/main.py --config /content/drive/MyDrive/SAMSLR/SL-GCN/config/sign/test/test_joint_motion.yaml
python /content/drive/MyDrive/SAMSLR/SL-GCN/main.py --config /content/drive/MyDrive/SAMSLR/SL-GCN/config/sign/test/test_bone_motion.yaml
python /content/drive/MyDrive/SAMSLR/ensemble/gcn/ensemble_wo_val.py
python /content/drive/MyDrive/SAMSLR/SSTCN/data_process/wholepose_features_extraction.py
python /content/drive/MyDrive/SAMSLR/SSTCN/test.py
python /content/drive/MyDrive/SAMSLR/ensemble/ensemble_multimodal_rgb.py


Writing /content/exec.sh


In [5]:
import subprocess
result = subprocess.run(["chmod","u+x","exec.sh"], stdout=subprocess.PIPE, text=True)
print(result.stdout)


In [6]:
%%writefile /content/drive/MyDrive/SAMSLR/app.py
import streamlit as st
import os
from IPython.display import HTML
from base64 import b64encode
import shutil
import pandas as pd
import subprocess
import streamlit.components.v1 as components
from googletrans import Translator
translator=Translator()

components.html(
    """
    <meta charset="utf-8">
    <meta name="viewport"
        content="width=device-width, initial-scale=1">
    <link rel="stylesheet" href=
"https://maxcdn.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css">
    <script src=
"https://ajax.googleapis.com/ajax/libs/jquery/3.3.1/jquery.min.js">
    </script>
    <script src=
"https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.14.7/umd/popper.min.js">
    </script>
    <script src=
"https://maxcdn.bootstrapcdn.com/bootstrap/4.3.1/js/bootstrap.min.js">
    </script>
</head>
    <div class="container">
        <div class="jumbotron" style="background-color:#100e2e;">
            <h1 style="color:#FFFFFF;">
                Sign Language Recognition
            </h1>
        </div>
    </div>
    """,
    height=200
)

st.write("Please Upload Video")
vid_file = st.file_uploader("Upload video", type=["mp4"])
# video_file = open('/content/drive/MyDrive/data/result_compressed.mp4', 'rb')
if vid_file!=None:
  with open(os.path.join("/content/drive/MyDrive/SAMSLR/input-video",vid_file.name),"wb") as f:
         f.write(vid_file.getbuffer())
  save_path = "/content/drive/MyDrive/SAMSLR/input-video/"+vid_file.name
  st.write(save_path)
  compressed_path = "/content/drive/MyDrive/SAMSLR/input-video/result_compressed.mp4"
  os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")
  video_file = open('/content/drive/MyDrive/SAMSLR/input-video/result_compressed.mp4', 'rb')
  video_bytes = video_file.read()
  st.video(video_bytes)
  os.system("rm /content/drive/MyDrive/SAMSLR/input-video/result_compressed.mp4")
  input_path = '/content/drive/MyDrive/input'
  result_path = '/content/drive/MyDrive/results'
  test_features_path = '/content/drive/MyDrive/results/test_features'

  # Get path of video to be classified
  video_path = save_path
  video_label_path = '/content/drive/MyDrive/data/val30_labels.csv'
  # Create input and results directories
  if not os.path.exists(input_path):
      os.makedirs(input_path)
  if not os.path.exists(result_path):
      os.makedirs(result_path)
  work_dir_path = '/content/drive/MyDrive/results/work_dir'
  if not os.path.exists(work_dir_path):
      os.makedirs(work_dir_path)
  if not os.path.exists(test_features_path):
      os.makedirs(test_features_path)
  # Copy to input folder
  shutil.copy(video_path,input_path+'/test_example_color.mp4')

  # Create csv for input file
  df=pd.read_csv(video_label_path)
  video_name = video_path.split('/')[-1][:-10]
  new_df = df.loc[(df['file_name']==video_name)]
  new_df.drop(['Unnamed: 0'], axis=1,inplace = True)
  print(new_df)
  new_df = new_df.values.tolist()
  new_df[0][0] = 'test_example'
  new_df = pd.DataFrame(new_df)
  new_df.columns = ['file_name','class_id']
  new_df.to_csv('/content/drive/MyDrive/results/test_labels.csv')


  # Create npy directory
  npy_path = result_path+'/npy3'
  if not os.path.exists(npy_path):
      os.makedirs(npy_path)

  st.write("Your video is being processed")
  
  st.write('-----------------------------------------------------')
  st.write('Generating skeleton npy features for input video')

  # From npy files, create joint data npy file
  st.write('-----------------------------------------------------')
  st.write('Generating Joint features')

  # From npy file, create bone data npy file
  st.write('-----------------------------------------------------')
  st.write('Generating Bone features')

  # From npy file, create motion data npy files for join and bone
  st.write('-----------------------------------------------------')
  st.write('Generating Joint & Bone motion features')

  # Create joint pkl file for ensemble
  st.write('-----------------------------------------------------')
  st.write('Creating joint pkl file for ensemble')

  # Create bone pkl file for ensemble
  st.write('-----------------------------------------------------')
  st.write('Creating bone pkl file for ensemble')

  # Create joint motion pkl file for ensemble
  st.write('-----------------------------------------------------')
  st.write('Creating joint motion pkl file for ensemble')

  # Create bone motion pkl file for ensemble
  st.write('-----------------------------------------------------')
  st.write('Creating bone motion pkl file for ensemble')

  # Create SLGCN ensemble
  st.write('-----------------------------------------------------')
  st.write('Generating score from SLGCN skeleton features')

# Create SSTCN features
  st.write('-----------------------------------------------------')
  st.write('Generating SSTCN features')

  # Generate Score from SSTCN
  st.write('-----------------------------------------------------')
  st.write('Generating score from SSTCN features')

  # Making Final Predictions
  st.write('-----------------------------------------------------')
  st.write('Making final predictions')

  result = subprocess.run("./exec.sh", stderr=subprocess.PIPE, text=True)
  print(result.stderr)


  # Print english output
  output = pd.read_csv('/content/drive/MyDrive/results/predictions_rgb.csv')
  if len(output)!=0:
    output = output.values.tolist()
    class_id = output[-1][-1]

    print('Class ID is : ',class_id)

    mapping = pd.read_csv('/content/drive/MyDrive/Class-ID Mapping.csv')
    mapping = mapping[mapping['ClassId']==class_id]
    mapping_output = mapping.values.tolist()
    english_word = mapping_output[-1][-1]
    sentence = ' '.join(english_word.split('_'))
    print('English word is :', sentence)
    output=pd.DataFrame(columns=['Unnamed: 0', 'file_name', 'class_id'])
    output.to_csv("/content/drive/MyDrive/results/predictions_rgb.csv",index=False)
    output='English word is : '+sentence
    st.title(output)
    result_kannada = translator.translate(sentence, dest='kn')
    result_hindi=translator.translate(sentence, dest='hi')
    kannada_output="Translated to Kannada: "+result_kannada.text
    hindi_output="Translated to Hindi: "+result_hindi.text
    st.title(kannada_output)
    st.title(hindi_output)
    
    
  # os.system("rm "+save_path)


Overwriting /content/drive/MyDrive/SAMSLR/app.py


In [ ]:
!streamlit run drive/MyDrive/SAMSLR/app.py & npx localtunnel --port=8501

2022-07-01 02:50:10.079 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 3.946s
your url is: https://social-sites-march-34-143-175-31.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.143.175.31:8501

ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-li

In [ ]:
#code playground